## LSB1 Algorithm

- **Raster Scan** : Le parcours de chaque pixel de l'image se fait ligne par ligne, de gauche à droite et de haut en bas.
- **Un pixel en blanc** : Un pixel dont les valeurs RGB sont toutes égales à 255, ce qui le rend blanc. (255, 255, 255)
- **verbose output** : Affichage détaillé des informations pendant l'exécution du code, utile pour le débogage et la compréhension du processus.
- **Alternative au raster scan** : flaten and reshape. Au deut on flaten l'image en 1D, puis on reshape en 2D (nombre de pixels, 3)

### Importer les bibliotheques

In [1]:
from PIL import Image
import numpy as np

#### 1/ Convertir l'image en tableau numpy de pixels (matrice)

In [2]:
image = Image.open("image.png").convert('RGB')
image_array = np.array(image)

In [3]:
print(image_array.shape)

(1500, 1000, 3)


**Signification** : L'image a une hauteur de 1500 pixels, une largeur de 1000 pixels et 3 canaux de couleur (Rouge, Vert, Bleu).

In [4]:
print(image_array[0][2])

[254 252 251]


#### 2/ Fonction qui convertit les pixels du tableau `image_array` en pixels avec des valeurs pairs

In [5]:
def get_image_even(image_array):
    return image_array - image_array % 2

In [6]:
print(f" Avant conversion : {image_array[0][1]}")
print(f" Apres conversion : {get_image_even(image_array)[0][1]}")

 Avant conversion : [254 252 251]
 Apres conversion : [254 252 250]


#### 3/ Fonction qui transforme le texte a coder en binare

- `ord()`: Renvoie le code ASCII d'un caractère donné.
- `chr()`: Renvoie le caractère correspondant à un code ASCII donné.
- `bin()`: Convertit un entier en une chaîne binaire.
- `zfill()`: Remplit une chaîne avec des zéros à gauche jusqu'à atteindre une longueur spécifiée.
**RQ** : 
    - Un octet est composé de 8 bits.
    - La table ascii possède 1_114_112 caractères.
    - On reconnait qu'un charactere est en binaire si il est prefixé par "0b".
    - On aurait pu choist la taille d'espac 8 car un caractetere est codé sur 8 bits, mais on choist 21 car la table ascii possède 1_114_112 caractères et 2^21 = 2_097_152 > 1_114_112 et 2^10 = 1_048_576 < 1_114_112
    - Donc 21 bits sont nécessaires pour coder tout les caractere ascii.

In [7]:
def convert_text_to_binary(text):
	bits_str = "".join([bin(ord(char))[2:].zfill(21) for char in text]) # Chaine de carcatère representant le binaire de text
	list_of_bits = [int(bit) for bit in bits_str]
	return list_of_bits

#### 4/ Fonction qui va modifier l'image pour y cacher le message binaire

- `Image.fromarray()`: Crée une image à partir d'un tableau numpy.
- `.save()`: Sauvegarde l'image modifiée dans un nouveau fichier.

In [8]:
def watermark_image(image_array, text):
	even_array_image = get_image_even(image_array)
	list_of_bits = convert_text_to_binary(text)

	number_rows, number_cols, number_canals = even_array_image.shape

	for row in range(0, number_rows):
		for col in range(0, number_cols):
			for canal in range(0, number_canals):
				if list_of_bits :
					even_array_image[row][col][canal] += list_of_bits.pop(0)

	if list_of_bits:
		print("Ecriture du message incomplète !")


	Image.fromarray(even_array_image).save('image_watermarked.png')

In [ ]:
def get_message_from_watermarked_image(image_array):
    initila_binary_message = image_array.flatten() % 2 # On aplati l'image en 1D et on prend le LSB de chaque pixel. On utilise %2 afin d'obtenir le LSB (0 ou 1) 
    print(initila_binary_message)

### Entrée du programme

- `Image.open()`: Ouvre une image à partir d'un fichier.
- `.convert()`: Convertit l'image dans un format (flag) spécifique (RGB, GRAY, etc.).
- `.save()`: Sauvegarde l'image modifiée dans un nouveau fichier.

In [33]:
if __name__ == "__main__":
	image = Image.open("image.png").convert('RGB')
	image_array = np.array(image)
	watermark_image(image_array, text="chocolat")


	image_watermarked = Image.open("image_watermarked.png").convert('RGB')
	image_watermarked_array = np.array(image_watermarked)
	print(type(get_message_from_watermarked_image(image_watermarked_array)))

	# Comme on sait que chaque caractère est codé sur 21 bits, et qu le message est au debut on peut l'extraire : 

	# On regroupe les bits par 21
	# On convertit chaque groupe de 21 bits en caractère avec chr(int(bits,
	# On reconstitue le message
	# Debut du code 
	# Fin du code
	

[0 0 0 ... 0 0 0]
<class 'NoneType'>


In [42]:
print(image_watermarked_array.shape)

(1500, 1000, 3)


In [45]:
print(image_watermarked_array[0][1])

[254 252 250]


In [46]:
print(image_watermarked_array)

[[[254 252 250]
  [254 252 250]
  [254 252 250]
  ...
  [240 150  36]
  [240 150  34]
  [240 150  34]]

 [[254 252 250]
  [254 252 250]
  [254 252 250]
  ...
  [240 150  36]
  [240 150  34]
  [240 150  34]]

 [[254 252 250]
  [254 252 250]
  [254 252 250]
  ...
  [240 150  36]
  [242 150  38]
  [242 150  38]]

 ...

 [[180  46  34]
  [180  46  34]
  [180  46  34]
  ...
  [ 88   2   0]
  [ 88   2   0]
  [ 88   2   0]]

 [[180  46  34]
  [180  46  34]
  [180  46  34]
  ...
  [ 90   2   0]
  [ 90   2   0]
  [ 90   2   0]]

 [[180  46  34]
  [180  46  34]
  [180  46  34]
  ...
  [ 90   2   0]
  [ 90   2   0]
  [ 90   2   0]]]


#### 5/ Cette fois, avant de mettre le dans l'image watermarked, on va le crypter avec le cryptage de vigenere

##### 5.1/ cesar cipher function

In [16]:
def cesar_cipher(text, key, cipher=True):

	key = key if cipher else -key # ternary
	
	crypted_text = ""
	for char in text:
		crypted_char = chr((ord(char) + key) % 1_114_112)
		crypted_text += crypted_char

	return crypted_text

##### 5.2/ vigenere cipher function

In [17]:
def vigenere_cipher(text, password, cipher=True):
	list_of_keys = [ord(char) for char in password]
	crypted_text = ""

	for index, char in enumerate(text):
		current_key = list_of_keys[index % len(list_of_keys)]
		crypted_char = cesar_cipher(text=char, key=current_key, cipher=cipher)
		crypted_text += crypted_char

	return crypted_text


In [19]:
def watermark_vigenere_cipher_cipher_image(image_array, text):
    crypted_text = vigenere_cipher(text=text, password="azerty12345", cipher=True)
    watermark_image(image_array, crypted_text)
    return crypted_text

In [ ]:
def get_vigenere_ciphered_message_from_watermarked_image(image_array):
    initila_binary_message = image_array.flatten() % 2 # On aplati l'image en 1D et on prend le LSB de chaque pixel. On utilise %2 afin d'obtenir le LSB (0 ou 1) 
    # lapartie du texte etant toujours au debut
    bits_str = "".join([str(bit) for bit in initila_binary_message])
    chars = [bits_str[i:i+21] for i in range(0, len(bits_str), 21)]
    message = ""
    for char in chars:
        message += chr(int(char, 2))    
    decrypted_message = vigenere_cipher(text=message, password="azerty12345", cipher=False)
    return decrypted_message

if __name__ == "__main__":
    image = Image.open("image.png").convert('RGB')
    image_array = np.array(image)
    crypted_text = watermark_vigenere_cipher_cipher_image(image_array, text="chocolat")


    image_watermarked = Image.open("image_watermarked.png").convert('RGB')
    image_watermarked_array = np.array(image_watermarked)
    print(get_vigenere_ciphered_message_from_watermarked_image(image_watermarked_array))

chocolat􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌􏿋􏾟􏾆􏾛􏾎􏾌􏾇􏿏􏿎􏿍􏿌